In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import geopandas as gpd
import urllib

# In Class Practice #8: Geopandas

### Overall objective of this practice: how many USGS gages are available for each HUC8 basin?
---
In this practice, we will learn how to use `geopandas` to read and generate shapefiles, conduct basic shapefile manipulations, and plot beautiful maps!</br>

# !!! Download the dataset first!!!

https://drive.google.com/drive/folders/1bH26iJFwFNSv_1ZJ8gwyktqqkfQXONQD?usp=drive_link

In [ ]:
# NOTE: Here we are using "gpd.read_file(filename)" to read in
# shapefiles

# read the geographic boundary of every counties in 
# NY york state
ny = gpd.read_file(
    'NY_counties.gpkg'
)

# read the shapefile files for all HUC-8 level watershed
# that touches New York State boundary
huc8 = gpd.read_file(
    'USGS_HUC8.NY_State.gpkg'
)

# read the text file that contains all USGS sites in 
# New York State
gages_df = pd.read_csv('gage_site.new_york_state.txt',
                        comment='#',sep='\t')


## 0. What does a shapefile look like after we read in it using geopandas?

Overall, if you take a look at what `ny` looks like, it is like a pandas framework! </br>
Maybe that's why it is called `geopandas` </br>

But what's the difference between normal `pandas.dataframe` versus `geopandas dataframe`?
* `geopandas dataframe` contains the column **geometry**, which is usually a shapefile, such as a point, a line, or a polygon.

In [ ]:
ny

## 0.1. Quick visualization
Additionally, you can also quickly visualize it using `plot` function

In [ ]:
ny.plot()

## 0.2. What's the existing projection for this shapefile?
We can use `ny.crs` to examine it. </br>
**CRS** is short for A projected coordinate reference system.</br>
This projection information looks not familiar! How can we project it to a crs that we are more familiar with? We will show you how to do it below

In [ ]:
ny.crs

## 1.1. Use lat/lon coordinates of USGS sites to generate point-type shapefiles

In [ ]:
# get rid of the first line 
gages_df = gages_df.iloc[1:]

# convert the latitude/longtiude to numbers
gages_df.loc['dec_lat_va',:] = gages_df['dec_lat_va'].astype(np.float32)
gages_df.loc['dec_long_va',:] = gages_df['dec_long_va'].astype(np.float32)

In [ ]:
# Create the shapefile of gage locations using latitude/longitude
# NOTE: In the coordinates for points, 
#       x-axis is longitude, y-axis is latitude.

gages = gpd.GeoDataFrame(
        gages_df, geometry=gpd.points_from_xy(gages_df['dec_long_va'], 
                                              gages_df['dec_lat_va']), crs = huc8.crs)


## 1.2. Get New York State boundary by "dissolving" the county-level boundaries

In [ ]:
#%%
# The various polygons in the New York State shapefile
# are boundaries for counties, and don't really
# mean anything as far as the hydrology of NY. So,
# let's get rid of them. In GIS-language this is called
# "dissolving" the polygons. 
#
# Here is to make the `ny` variable be a 
# geodataframe with only a single geometry.

ny = ny.dissolve()
ny.plot()

## 1.3. What if we have USGS gage info across the entire US? How can we quickly select the sites within a shapefile?

In [ ]:
#%%
# Pull out only the gages in NY from 
# the `gages` dataset, save this as `ny_gages`
# In GIS-language this is called "clipping" 

ny_gages = gages.clip(ny)
ny_gages.plot()


## Oh no! The code above does not work? Why is that?
The CRS of `ny_gages` and `ny` are different! So, let's convert the `CRS` of `ny` 

In [ ]:
ny = ny.to_crs(huc8.crs)

### Let's try it one more time!

In [ ]:
ny_gages = gages.clip(ny)
ny_gages.plot()

## 1.4. Basic visualization!

In [ ]:
# Make a plot showing NY State in "lightgrey"
# and the locations of the gages in Arizona plotted as
# "crimson" colored points.
# NOTE: Calling `.plot` on a geodataframe will return 
#       a new axis object which can be passed to 
#       subsequent plot commands 

# NOTE: You might try setting `markersize=3` or similar
#       when you are plotting the gages, so that it's 
#       easier to see them.

ax = ny.plot(color='lightgrey')
ny_gages.plot(color='crimson', ax=ax, markersize=3)

## 1.4.1. What if we want to change the CRS of this map?

Here we can introduce the package `cartopy`, a Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses. </br>
The function `cartopy.crs` contains a large collection of CRS, that should suffice most plotting needs!</br>
In addition, we will also use `pyproj`, a Python interface to PROJ (cartographic projections and coordinate transformations library).




In [ ]:
import cartopy.crs as ccrs
import pyproj

In [ ]:
plt.figure(dpi=150)

# 1. define the targeted CRS
proj1 = ccrs.Mercator(central_longitude=-76)

# 2. create an `plt.axes` using the targeted projection information
ax = plt.axes(projection=proj1)

# 3. convert the shapefile to the targeted CRS
# Let me explain a bit more about the options in 
# plotting function
# zorder: the rank of each layer. Layers with lower number means this
#         layer can possibly be covered by other layers
# linewidth: the line width of boundary
# we can specify both edge color and facecolors for the polygons!
# If we want transparent, we can specify the number as "none".
ny.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                               zorder=0,
                                               linewidth=2, 
                                               edgecolor='none',
                                               facecolor='lightgrey')
ny_gages.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                     zorder=1,
                                                     facecolor='crimson', 
                                                     edgecolor='yellow',
                                                     linewidth=0.6,
                                                     markersize=8)

# here we can also add the coastlines in the basemap
ax.coastlines(linewidth=0.5, color='k', zorder=1)


## What if we change the CRS to `Plate Carree`?

In [ ]:
proj2 = ccrs.PlateCarree()

In [ ]:
plt.figure(dpi=150)
# note: from here, we changed the CRS for all relevant codes
#       proj1 -> proj2
ax = plt.axes(projection=proj2)
ny.to_crs(pyproj.CRS(proj2.proj4_params)).plot(ax=ax,
                                               zorder=0,
                                               linewidth=2, 
                                               edgecolor='none',
                                               facecolor='lightgrey')
ny_gages.to_crs(pyproj.CRS(proj2.proj4_params)).plot(ax=ax,
                                                     zorder=1,
                                                     facecolor='crimson', 
                                                     edgecolor='yellow',
                                                     linewidth=0.6,
                                                     markersize=8)
ax.coastlines(linewidth=0.5, color='k', zorder=1)

### Compare this map with the previous one, did you see any differences?
### Which one do you prefer more?

In all exercise below, we will stick with the Mercator `proj1`

## 1.5. Practice add HUC 8 boundaries in the map!

In [ ]:
# %%
# I also gave you a dataset of watershed
# boundaries (called HUCs, for hydrologic unit code).
# I gave you the "level 8" units, where a smaller unit
# level means a larger spatial aggregation, and a larger
# code is more fine-scaled. This is stored in the variable 
# `huc8`. 
#
# Bottom layer: the huc8 boundaries with facecolor in"lightgrey",
#               edgecolor in "white"
# Middle layer: the black outline of New York State Boundary "ny" on top
#               of the bottom layer, with "transparent" facecolor.
# Top layer: plot the gages contained in New York again as "crimson"
# points (no need to add yellow boundaries)
# 
# We will use "zorders" to decide the layer orders

## 1.6. Can we highlight the location of Niagara River gage?

### 1.6.1. Locate the niagara_gage using station names

In [ ]:
# For this step, I want you to plot the location
# of the Niagara River gage that we've been using as an example. 
# 
# To do this, first find the row where # the `'station_nm'` 
# column of `ny_gages` is equal to # the `name` variable. 
# Then use that information to select out only the Niagara
# river gage into the variable `niagara_gage`.
#
# The resulting plot should put a big star where the 
# gage location is. All other gages in New York will
# still appear as dots.
name = "NIAGARA RIVER AT BUFFALO NY"
is_the_gage = ny_gages['station_nm'] == name

# Found solution : 
#  https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
niagara_gage = ny_gages.loc[is_the_gage]

### 1.6.2. Plot

In [ ]:
# Plotting code
# This could be useful for your future project
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)
ny.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                               zorder=1,
                                               linewidth=2, 
                                               edgecolor='k',
                                               facecolor='none')
ny_gages.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                     zorder=2,
                                                     facecolor='rosybrown', 
                                                     edgecolor='none',
                                                     linewidth=0.6,
                                                     markersize=8)
huc8.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                     zorder=0,
                                                     facecolor='lightgrey', 
                                                     edgecolor='white',
                                                     linewidth=0.6)

# Note: here we can customize the marker
# marker: this can be used to change the scatter shape
# markersize: this can be used to change the size of the scatter
niagara_gage.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                         zorder=3,
                                                         marker='*',
                                                         markersize=100,
                                                         facecolor='crimson', 
                                                         edgecolor='white',
                                                         linewidth=0.6)
ax.coastlines(linewidth=0.5, color='k', zorder=1)


## 1.7. Let's combine the knowledge about downloading streamflow data from USGS!
### 1.7.1. Summarize the code for downloading USGS datasets into a function!

In [ ]:
#%%
# Now let's combine this with our knowledge
# about downloading streamflow data from USGS!
# 
# I've provided you with the functions for downloading
# data that we've used in the past. You don't have to
# do anything for this step.
def create_usgs_url(site_no, begin_date, end_date):
    return (
        f'https://waterdata.usgs.gov/nwis/dv?'
        f'cb_00060=on&format=rdb&referred_module=sw&'
        f'site_no={site_no}&'
        f'begin_date={begin_date}&'
        f'end_date={end_date}'
    )

def open_usgs_data(site, begin_date, end_date):
    url = create_usgs_url((site), begin_date, end_date)
    response = urllib.request.urlopen(url)
    df = pd.read_table(
        response,
        comment='#',
        sep='\s+',
        names=['agency', 'site', 'date', 'streamflow', 'quality_flag'],
        index_col=2,
        parse_dates=True,
        date_format='yyyy-mm-dd',
        engine='python'
    ).iloc[2:]

    # Now convert the streamflow data to floats and
    # the index to datetimes. When processing raw data
    # it's common to have to do some extra postprocessing
    df['streamflow'] = df['streamflow'].astype(np.float64)
    df.index = pd.DatetimeIndex(df.index)
    return df


### 1.7.2. Download the flow data for Niagara River

In [ ]:
#%%
# Now pull out the site id from the `niagara_gage`
# variable. This is contained in the `'site_no'` column, which
# stands for "Station ID". Put this into the variable 
# `station_id`
#
# Success on this one shoul just print out the first 5
# streamflow values for the Niagara River in Buffalo NY.
begin_date = '2012-10-01'
end_date = '2023-09-30'

station_id = niagara_gage['site_no']

site = station_id.values[0]
niagara_df = open_usgs_data(site, begin_date, end_date)
niagara_df.head()

### 1.7.3. Practice: Download the flow data for Cayuga River

In [ ]:
#%% 
# Now try pulling out a different gage location
# using it's name and download the USGS data for the 
# same time period as the `niagara_df`. Put this one in
# `other_gage_df`. Compare the two location's mean
# streamflows by printing them out.

# The station name for Cayuga River is: 
#        "CAYUGA CREEK NEAR LANCASTER NY"

# Example code from 1.6.2.: to locate Cayuga River gage
station_name = 'CAYUGA CREEK NEAR LANCASTER NY'
# TODO: INSERT your code here

# Example code from 1.7.2.: to download the data
# TODO: INSERT your code here

# print the mean flow
print("The mean flow for %s is %0.2f cfs"%(name,niagara_df['streamflow'].mean()))
print("The mean flow for %s is %0.2f cfs"%(station_name,other_gage_df['streamflow'].mean()))


## 1.8. How can we count the number of gages within each HUC8 basin?

In [ ]:
# From our original plots of the spatial
# distribution of gages it was clear that surface
# water access in New York is uneven. For this 
# exercise I want you to count the number of gages
# in New York for each of the HUC8 boundaries. 
# 
# To do this you'll have to iterate over the `huc8`
# variable using the `huc8.iterrows()` function, which
# basically loops over each row of the dataframe. 
# Instead of giving you just the row, it also gives 
# you the row column, which is why I have put `i, huc`
# in the for loop. `i` will keep track of the row number
# and `huc` will be the actual data from the row.
#
# I've got you started on the loop, but your next step
# is to "clip" from `az_gages` the "geometry" from the 
# `huc`. Then, count how many gages are in this selection
# by using the `len` function. Append the result of this
# to the `number_gages_in_huc` list.

number_gages_in_huc = []
for i, huc in huc8.iterrows():
    clipped_gages = ny_gages.clip(huc.geometry)
    #print(i, huc['name'], len(clipped_gages))
    number_gages_in_huc.append(len(clipped_gages))

huc8['number_gauges'] = number_gages_in_huc

In [ ]:
# Finally, plot the number of gages in
# each HUC - and don't forget to set `add_legend=True`!
# Use the colormap "Blues", and also plot the Arizona
# outline on top
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)

huc8.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                 column='number_gauges', 
                                                 cmap='Blues', 
                                                 legend=True)
ny.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax, 
                                               facecolor='none', 
                                               edgecolor='black')

# 2. A little bit more fancier plotting exercise

### 2.1. Discrete colormaps?
We can clearly see that the color distributions in Exercises are dominated by one outlier. </br>
So we would like to create a discrete colormaps using uneven distributed bins! </br>
How can we do it?

In [ ]:
# let's first look at the distribution of gauge numbers
# we plan to use histogram to look at it
plt.hist(huc8['number_gauges'])

In [ ]:
import matplotlib as mpl

# first, let's stick with the blues colormap
cmap_blues = mpl.colormaps.get_cmap('Blues')

# second, let's define the bins
# sometimes, we do not need to know the exact number of gages,
# we tend to know the ranges for each region
bounds_num_gage = [0,1,5,10,20,30,50,70]

# let's define the colors for each bin
# first of all, we can use camp_blues(number) to select 
#               colors within each colormap!
#               number ranges from 0 to 1. 
# Second, number of colors = len(bounds_num_gage)-1
color_num_gage_list = [cmap_blues((i+1)/len(bounds_num_gage)) for i in range(len(bounds_num_gage)-1)]

# assign the colors for each bin
cmap_num_gage = mpl.colors.ListedColormap(color_num_gage_list)
norm_num_gage = mpl.colors.BoundaryNorm(bounds_num_gage, len(color_num_gage_list))


In [ ]:
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)

huc8.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                 column='number_gauges', 
                                                 cmap=cmap_num_gage,
                                                 norm=norm_num_gage,
                                                 legend=True,
                                                 edgecolor='gray',
                                                 linewidth=0.4)
ny.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax, 
                                               facecolor='none', 
                                               edgecolor='black')
ax.coastlines(linewidth=0.5, color='k', zorder=1)


## 2.2. Add gridlines

In [ ]:
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LongitudeFormatter, LatitudeFormatter

In [ ]:
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)

huc8.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                 column='number_gauges', 
                                                 cmap=cmap_num_gage,
                                                 norm=norm_num_gage,
                                                 legend=True,
                                                 edgecolor='gray',
                                                 linewidth=0.4)
ny.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax, 
                                               facecolor='none', 
                                               edgecolor='black')
ax.coastlines(linewidth=0.5, color='k', zorder=1)

# Codes above are from Section 2.1.
# Codes below are for creating the gridlines
# dms: When default longitude and latitude locators and formatters are
#      used, ticks are able to stop on minutes and seconds if minutes is
#      set to True, and not fraction of degrees.
gl = ax.gridlines(draw_labels=True,
                  dms=True,
                  x_inline=False, 
                  y_inline=False,
                  linewidth=0.5, 
                  color='k', 
                  alpha=0.5, 
                  linestyle='--', 
                  zorder=4)

# decide the locations of the latitude/longitude labels will be 
# in top/bottom or left/right.
gl.top_labels = False
gl.right_labels = False
gl.left_labels = True
gl.bottom_labels = True

# The specific lat/lon grid lines
gl.xlocator = mticker.FixedLocator([-84,-80,-76,-72])
gl.ylocator = mticker.FixedLocator([40,42,44,46])

# whether we want to rotate the lat/lon label
gl.ylabel_style = {'rotation': 0}
gl.xlabel_style = {'rotation': 0}

## 2.3. What is we want to add some background (basemap) for the map?

Here we can simply using `ax.stock_img()`, which is the default basemap that is embedded in the package `cartopy`.

However, it is important to note that we will need to define the extent of the map using `set_extent`, when we use `ax.stock_img()` option. Otherwise, it will plot the map for entire globe.

In [ ]:
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)

# load the backgruond map
ax.stock_img()

# set the extent of the map
# [lon_min, lon_max, lat_min, lat_max]
ax.set_extent([-84,-71,39,47])

# below codes are identical to codes in Section 2.2.
huc8.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                 column='number_gauges', 
                                                 cmap=cmap_num_gage,
                                                 norm=norm_num_gage,
                                                 legend=True,
                                                 edgecolor='gray',
                                                 linewidth=0.4)
ny.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax, 
                                               facecolor='none', 
                                               edgecolor='black')
ax.coastlines(linewidth=0.5, color='k', zorder=1)


## 2.4. The basemap resolution is so low! Maybe we should just specify one green-ish color for land and one blue-ish color for the ocean!

Here we used a cartopy function, called `cartopy.feature`.

To simplify some very common cases, some pre-defined Features exist as cartopy.feature constants. The pre-defined Features are all small-scale (1:110m) Natural Earth datasets, and can be added with methods such as GeoAxes.add_feature:

**Name      (Description)**

cartopy.feature.BORDERS *(Country boundaries.)*

cartopy.feature.COASTLINE *(Coastline, including major islands.)*

cartopy.feature.LAKES *(Natural and artificial lakes.)*

cartopy.feature.LAND *(Land polygons, including major islands.)*

cartopy.feature.OCEAN *(Ocean polygons.)*

cartopy.feature.RIVERS *(Single-line drainages, including lake centerlines.)*


In [ ]:
import cartopy.feature as cfeature

In [ ]:
plt.figure(dpi=150)
ax = plt.axes(projection=proj1)
# below codes are identical to codes in Section 2.2.
huc8.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax,
                                                 column='number_gauges', 
                                                 cmap=cmap_num_gage,
                                                 norm=norm_num_gage,
                                                 legend=True,
                                                 edgecolor='gray',
                                                 linewidth=0.4, zorder=2)
ny.to_crs(pyproj.CRS(proj1.proj4_params)).plot(ax=ax, 
                                               facecolor='none', 
                                               edgecolor='black', zorder=3)

ax.coastlines(linewidth=0.5, color='k', zorder=4)

# Here we added colors for land and ocean
ax.add_feature(cfeature.LAND,zorder=1,facecolor="#E4E5BB")
ax.add_feature(cfeature.OCEAN,zorder=1,facecolor="#5E819D")

### final remarks: please carefully review the order of each layer (zorder) above. Does this rank make sense to you?